# Flux d'octets

Un Flux (**Stream** en Anglais) est une vue sur une suite d'Octets (`byte[]`).   
Dans la suite de ce Notebook, nous allons principalement parler de flux de fichiers `FileStream`, mais comme un flux est générique, les éléments que nous allons aborder s'appliquent également à d'autres formes de flux, telles que les `NetworkStream`, `MemoryStream`, `CryptoStream`, ...   

## De Byte à String

Une chaine de caractères est ... une suite de caractères ;)   
Mais un caractère ne tient pas forcément sur 1 `Byte`, ça va dépendre de **l'encodage** utilisé.  
L'encodage le plus ancien c'est **ASCII**: Il tient sur 1 Octet, et est suffisant pour les 26 lettres de l'alphabet occidental, avec les majuscules, les minuscules, les chiffres, les caractères spéciaux.   
Mais pour supporter plus de possiblités (Cyrillique, Kanjii, Hangul, ...), on a crée d'autres formes d'encodage tels que **Unicode**, **UTF-8**, **UTF-16**, ... Or en **UTF-8** par exemple, l'encodage d'un caractère peut se faire sur 1 à 4 Octets !! Il est donc **impossible** de lire convertir directement 1 Caractère en 1 Octet.

En C#, il existe des **encoders** qui vont nous permettre de passer de l'un à l'autre, mais celà sous entend que vous savez ce que vous manipulez.  

> Par exemple, on peut passer d'une chaine à un tableau d'octets...


In [ ]:
string chaine = "Bonjour à tous ! Voici quelques caractères internationaux : é, ô, ü, ß";
ASCIIEncoding ascii = new ASCIIEncoding();
UTF8Encoding utf8 = new UTF8Encoding();
byte[] infoAscii = ascii.GetBytes(chaine);
byte[] infoUtf8 = utf8.GetBytes(chaine);

Console.WriteLine( $"En ASCII, nous avons {infoAscii.Length} octets");
Console.WriteLine( $"En UTF8, nous avons {infoUtf8.Length} octets");

> ... et d'un tableau d'octets à une chaine   

Pour cela, il va falloir préciser l'**encoder**, le tableau contenant les infos, mais aussi, la position dans le tableau à laquelle on démarre, et sur combien d'octets.   
On peut aussi tout simplement donner le tableau d'octets, et le C# décodera l'ensemble.

In [ ]:
string nouvelleChaine = ascii.GetString( infoAscii, 0, infoAscii.Length );
Console.WriteLine( nouvelleChaine );

nouvelleChaine = utf8.GetString( infoUtf8 );
Console.WriteLine( nouvelleChaine );


## Ecriture dans un flux

On va écrire des Octets qui, comme pour les manipulations précédentes, peuvent être issus de chaines de caractères après conversion.

In [ ]:
using System.IO;
// Notre fichier Test
string path = @"c:\temp\MyTest.txt";
// On supprime s'il existe
if (File.Exists(path))
{
    File.Delete(path);
}
// Creation du fichier et fabrication du flux.
FileStream flux = File.Create(path);

string duTexte = "Voilà un peu de texte";
UTF8Encoding utf8 = new UTF8Encoding();
byte[] infoUtf8 = utf8.GetBytes(duTexte);
flux.Write( infoUtf8 );
// !!! Attention !!! N'oubliez pas de fermer le flux quand vous avez terminé !!!
flux.Close();


## Ouverture / Fermeture

Un problème que l'on rencontre souvent, est l'oubli de la fermeture du flux. Dans une telle situation, c'est le **Garbage Collector** (**Ramasse Miettes** en français) qui va décider du moment de fermeture, et ce sera au moment où la variable sera supprimée de la mémoire, donc un moment que nous ne maitrisons pas.

> Essayez à nouveau le code précédent, mais mettez d'abord en commentaires la fermeture.  
A la première execution vous ne verrez pas de changement, mais si vous essayez une deuxième fois, quelles sont les conséquences ?
Vous allez surement devoir fermer VSCode pour passer à la suite....

## Durée de vie d'une variable : instruction `using`

L’instruction `using` fournit une syntaxe pratique qui garantit l’utilisation correcte d’objets en précisant leur durée de vie.   

> Le code ci-dessous remplit la même fonction que le précédent, mais en précisant quand _créer_ et _supprimer_ la variable de flux.   
> La _suppression_ du flux entraine la fermeture du flux, car la classe `FileStream` est écrite comme cela.

In [ ]:
using System.IO;
// Notre fichier Test
string path = @"c:\temp\MyTest.txt";
// On supprime s'il existe
if (File.Exists(path))
{
    File.Delete(path);
}
// Creation du fichier et fabrication du flux.
using ( FileStream flux = File.Create(path) )
{
    string duTexte = "Voilà un peu de texte";

    UTF8Encoding utf8 = new UTF8Encoding();
    byte[] infoUtf8 = utf8.GetBytes(duTexte);
    flux.Write( infoUtf8 );
}
// 

# Lecture d'un flux

Et maintenant, comment lire un flux ?

On va tout d'abord l'ouvrir en précisant son chemin d'accès mais également son mode d'ouverture, via l'énumération [FileMode](https://learn.microsoft.com/fr-fr/dotnet/api/system.io.filemode?view=net-7.0).

In [ ]:
FileStream flux = File.Open(path, FileMode.Open );

On peut maintenant récupérer la taille du flux, réserver un tableau d'octets, pour y stocker l'ensemble des octets.

In [ ]:
var taille = flux.Length;
byte[] contenu = new byte[taille];
flux.Read( contenu, 0, (int)taille );
Console.WriteLine( $"{taille} Octets lus.");

Sans oublier de fermer le flux...

In [ ]:
flux.Close();

Il est à noter que comme on lit le flux d'octets, on avance dans le fichier.  
Le **pointeur de fichier** mémorise où on se trouve dans le fichier, et il est possible de l'avancer comme on le souhaite à l'aide de `Seek`.

> Ainsi, on va se positionner sur les 5 derniers octets de fichiers, avant de les lire et d'afficher leur valeur en Hexadecimal, Décimal et Caractère.   
> On remarquera le contenu de la chaine d'interpolation : `{index[,alignment][:formatString]}`

In [ ]:
FileStream flux = File.Open(path, FileMode.Open );
var taille = flux.Length;
if ( taille > 6 )
    flux.Seek( taille - 6, SeekOrigin.Begin );
byte[] contenu = new byte[6];
flux.Read( contenu, 0, contenu.Length );
flux.Close();

foreach( byte b in contenu)
{
    Console.WriteLine( $"0x{b:X2} - {b,3} - {(char)b}");
}

Et on peut aussi également lire le contenu complet du fichier en une seule fois.

In [ ]:
var contenu = File.ReadAllBytes( path );
Console.WriteLine( $"{contenu.Length} Octets lus.");

# Terrain de jeu: A vous de jouer

Réaliser un programme qui peut lire un fichier texte, et le reécrire à l'envers.   
> 1. Le nouveau fichier aura le nom du fichier d'orgine, mais l'envers. (Y compris l'extension)
> 2. Le contenu du nouveau fichier sera le même que celui d'origine, mais à l'envers...caractère par caractère.
> 3. Afficher le contenu du nouveau fichier.
> 4. Une fois la copie réalisée, écrivez le code permettant de vérifier que les fichiers sont effectivement "symétriques".


In [ ]:
Console.WriteLine("Playground");